In [ ]:
#Open API Fine Tuning Dashboard URL: https://platform.openai.com/finetune/

In [1]:
from dotenv import load_dotenv
from openai import OpenAI
import os

# --- Load API Key ---
load_dotenv(override=True, dotenv_path="../.env.local")
my_api_key = os.getenv("OPENAI_API_KEY")


client = OpenAI(api_key=my_api_key)

In [2]:
with open("data/training_data.jsonl", "rb") as f:
    training_file_obj = client.files.create(file=f, purpose="fine-tune")

print("Uploaded training file ID:", training_file_obj.id)

with open("data/validation_data.jsonl", "rb") as f:
    validation_file_obj = client.files.create(file=f, purpose="fine-tune")

print("Uploaded validation file ID:", validation_file_obj.id)

Uploaded training file ID: file-VuChw5U2rG6t1Jaaxoyqtm
Uploaded validation file ID: file-NLRfect6AniRNkrkXdokch


In [5]:
# Create a fine-tuning job
job = client.fine_tuning.jobs.create(
    training_file=training_file_obj.id,       # The file ID you uploaded
    validation_file=validation_file_obj.id,     # Optional
    model="gpt-4.1-nano-2025-04-14",               # or "gpt-3.5-turbo", etc.
    suffix="Yamaha-customer-support_SLC1"       # Optional model name suffix
)

print("Fine-tune job created:", job.id)

Fine-tune job created: ftjob-ANMRNlEc82bXqYUOaZKtahX3


In [6]:
jobs = client.fine_tuning.jobs.list(limit=5)
for j in jobs.data:
    print(j.id, j.status, j.fine_tuned_model)

ftjob-ANMRNlEc82bXqYUOaZKtahX3 validating_files None


In [ ]:
def ask_question_without_finetuning(prompt):
    print(f"User asked: {prompt}")
    response = client.chat.completions.create(
        model="gpt-5-nano",
        messages=[
            {"role": "system", "content": "You are a helpful assistant. Answer as concisely as possible."},
            {"role": "user", "content": prompt}
        ]
    )
    print (response)
    return response.choices[0].message.content  

def ask_question_with_finetuning(prompt):
    print(f"User asked: {prompt}")
    response = client.chat.completions.create(
        model="ft:gpt-4.1-2025-04-14:personal:brand-voice-support:CPjcskrj",
        messages=[
            {"role": "system", "content": "You are a helpful assistant. Answer as concisely as possible."},
            {"role": "user", "content": prompt}
        ]
    )
    print (response)
    return response.choices[0].message.content  

In [ ]:
import time
while True:
    # Ask user for a question
    user_prompt = input("Ask something: ")

    if (user_prompt.lower() != 'quit'):
        # Get and print the response
        response = ask_question_without_finetuning(user_prompt)
        print("\n[WITHOUT FINE TUNING] OpenAI says:", response)

        response = ask_question_with_finetuning(user_prompt)
        print("\n[WITH FINE TUNING] OpenAI says:", response)

        # add delay of 3 seconds
        time.sleep(3)
    else:
        break    